In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyaudio
import matplotlib.animation as animation

import paho.mqtt.publish as publish

mqtt_broker = "192.168.1.6"
topic = "your-topic"
message = "Hello, ESP32!"

sendCount = 100

# Set up parameters
duration = 10  # Total duration for audio capture (in seconds)
sample_rate = 44100  # Sample rate
chunk_size = 1024  # Number of frames per buffer
frequency_bin_width = 100  # Frequency bin width in Hz

# Initialize PyAudio
p = pyaudio.PyAudio()

# Open stream
stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=sample_rate,
                input=True,
                frames_per_buffer=chunk_size)

frequencies = np.fft.fftfreq(chunk_size, d=1/sample_rate)

# Find the maximum magnitude in each 100 Hz frequency bin
max_magnitudes = [1] * len(range(0, len(frequencies), frequency_bin_width))
max_magnitude_max = [1] * len(range(0, len(frequencies), frequency_bin_width))
max_magnitude_percent = [1] * len(range(0, len(frequencies), frequency_bin_width))
    
while(1):
    audio_data = np.frombuffer(stream.read(chunk_size), dtype=np.int16)
    fft_result = np.fft.fft(audio_data)
    fft_magnitude = np.abs(fft_result)
    
    for i in range(0, len(frequencies), frequency_bin_width):
        index = int(i/frequency_bin_width)
        bin_magnitudes = fft_magnitude[i:i+frequency_bin_width]
        max_magnitude = np.max(bin_magnitudes)
        max_magnitudes[index] = max_magnitude
        if(max_magnitude>max_magnitude_max[index]):
            max_magnitude_max[index] = max_magnitude
            
        max_magnitude_percent[index] = (max_magnitudes[index]/max_magnitude_max[index])*100
        
#     print(max_magnitude_percent[10])
#     if(0 == sendCount):
#         sendCount = 100
        message = str(max_magnitude_percent[10])
        publish.single(topic, message, hostname=mqtt_broker)

#     sendCount -= 1
        